In [ ]:
!pip install groq
!pip install dspy-ai[qdrant]
!pip install langchain
!pip install sentence_transformers
!pip install langchain_community
!pip install pypdf

In [12]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=400,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)
docs = PyPDFLoader("/content/wordWarData.pdf").load_and_split(text_splitter = text_splitter )


doc_contents = [doc.page_content for doc in docs]


# List to hold the IDs for each document
doc_ids = list(range(1, len(docs) + 1))


In [14]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2", device='cpu')
vectors = model.encode(doc_contents)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [35]:
import dspy

lm = dspy.OllamaLocal(model="llama3",timeout_s = 180)



In [16]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

client = QdrantClient(":memory:")

In [17]:
client.recreate_collection(
    collection_name="cf_data",
    vectors_config=VectorParams(size=768, distance=Distance.COSINE),
)

<ipython-input-17-bdf578095156>:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [18]:
client.upload_collection(
    collection_name="cf_data",
    ids=[i for i in range(len(doc_contents))],
    vectors=vectors,

)

In [19]:
from dspy.retrieve.qdrant_rm import QdrantRM
qdrant_retriever_model = QdrantRM("cf_data", client, k=3)

100%|██████████| 77.7M/77.7M [00:03<00:00, 22.3MiB/s]


In [20]:
dspy.settings.configure(rm=qdrant_retriever_model, lm=turbo)


In [21]:
def get_context(text):
    query_vector = model.encode(text)

    hits = client.search(
        collection_name="cf_data",
        query_vector=query_vector,
        limit=3
    )

In [22]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often 1 to 5 word answer")

In [23]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question):
        context = get_context(question)
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

In [24]:
rag = RAG()
def respond(query):
    response = rag(query)
    return response.answer

print(respond("What was the name of the German battleship sunk at the Battle of Jutland?"))

Reasoning: Let's think step by step in order to identify the German battleship sunk at the Battle of Jutland. We know that the battle took place on May 31, 1916, and several German ships were sunk...

Answer: Pommern.


In [25]:
import json
import dspy
traindata = json.load(open("/content/trainingData.json", "r"))['examples']
trainset = [dspy.Example(question=e['question'], answer=e['answer']).with_inputs('question') for e in traindata]

testdata = json.load(open("/content/testingData.json", "r"))['examples']
testset = [dspy.Example(question=e['question'], answer=e['answer']).with_inputs('question') for e in testdata]

In [26]:
testdata[0:2]

[{'question': 'Which battle in World War 2 marked the Allied liberation of Paris from German occupation?',
  'answer': 'Battle of Normandy'},
 {'question': 'Who was the leader of Norway during its occupation by Germany in World War 2?',
  'answer': 'Vidkun Quisling'}]

In [27]:
from dspy.evaluate.evaluate import Evaluate

# Set up the `evaluate_on_hotpotqa` function. We'll use this many times below.
evaluate_on_hotpotqa = Evaluate(devset=testset, num_threads=1, display_progress=True, display_table=17)

# Evaluate the `compiled_rag` program with the `answer_exact_match` metric.
metric = dspy.evaluate.answer_exact_match
evaluate_on_hotpotqa(rag, metric=metric)

Average Metric: 7 / 17  (41.2): 100%|██████████| 17/17 [00:11<00:00,  1.43it/s]


,question,example_answer,context,pred_answer,answer_exact_match
0,Which battle in World War 2 marked the Allied liberation of Paris from German occupation?,Battle of Normandy,None,Battle of Paris,False
1,Who was the leader of Norway during its occupation by Germany in World War 2?,Vidkun Quisling,None,Reasoning: Let's think step by step in order to identify the leader of Norway during this period. We need to consider the government of Norway...,False
2,What was the name of the Allied campaign to recapture the Philippines from Japanese forces in World War 2?,Battle of Leyte Gulf,None,"Reasoning: Let's think step by step in order to recall the major military operations of World War 2. We know that the Allies, led by...",False
3,Which battle in World War 2 marked the Soviet defeat of German forces in the Battle of Berlin?,Battle of Berlin,None,Battle of Berlin.,✔️ [True]
4,Who was the leader of Belgium during its occupation by Germany in World War 2?,Hubert Pierlot,None,Reasoning: Let's think step by step in order to identify the leader of Belgium during this period. We already know the answer from the context....,False
5,What was the name of the German defensive line in Italy during World War 2?,Gustav Line,None,Gustav Line,✔️ [True]
6,Which battle in World War 2 marked the Allied victory in North Africa?,Battle of El Alamein,None,El Alamein.,False
7,Who was the leader of the Netherlands during its occupation by Germany in World War 2?,Queen Wilhelmina,None,Reasoning: Let's think step by step in order to identify the leader of the Netherlands during this period. We need to consider the monarch who...,False
8,What was the name of the German offensive in the Ardennes Forest in World War 2?,Battle of the Bulge,None,Battle of the Bulge.,✔️ [True]
9,Which battle in World War 2 marked the Japanese defeat at the hands of the United States Navy?,Battle of Midway,None,Battle of Midway,✔️ [True]


41.18

In [28]:
from dspy import teleprompt

def validate_context_and_answer(example, pred, trace=None):
    if pred.context is None:
        return False
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM

teleprompter = teleprompt.BootstrapFewShot(metric=validate_context_and_answer)

compiled_rag = teleprompter.compile(RAG(), trainset=trainset)

100%|██████████| 31/31 [03:24<00:00,  6.59s/it]


In [29]:
from dspy.evaluate.evaluate import Evaluate

# Set up the `evaluate_on_hotpotqa` function. We'll use this many times below.
evaluate_on_hotpotqa = Evaluate(devset=testset, num_threads=1, display_progress=True, display_table=17)

# Evaluate the `compiled_rag` program with the `answer_exact_match` metric.
metric = dspy.evaluate.answer_exact_match
evaluate_on_hotpotqa(compiled_rag, metric=metric)

Average Metric: 9 / 17  (52.9): 100%|██████████| 17/17 [01:09<00:00,  4.11s/it]


,question,example_answer,context,pred_answer,answer_exact_match
0,Which battle in World War 2 marked the Allied liberation of Paris from German occupation?,Battle of Normandy,None,Liberation of Paris,False
1,Who was the leader of Norway during its occupation by Germany in World War 2?,Vidkun Quisling,None,King Haakon VII,False
2,What was the name of the Allied campaign to recapture the Philippines from Japanese forces in World War 2?,Battle of Leyte Gulf,None,Reasoning: Let's think step by step in order to identify the Allied campaign. We know that the campaign was a major operation in the Pacific...,False
3,Which battle in World War 2 marked the Soviet defeat of German forces in the Battle of Berlin?,Battle of Berlin,None,Battle of Berlin,✔️ [True]
4,Who was the leader of Belgium during its occupation by Germany in World War 2?,Hubert Pierlot,None,Reasoning: Let's think step by step in order to identify the leader of Belgium during its occupation by Germany in World War 2. We already...,False
5,What was the name of the German defensive line in Italy during World War 2?,Gustav Line,None,Gustav Line,✔️ [True]
6,Which battle in World War 2 marked the Allied victory in North Africa?,Battle of El Alamein,None,Battle of El Alamein,✔️ [True]
7,Who was the leader of the Netherlands during its occupation by Germany in World War 2?,Queen Wilhelmina,None,Queen Wilhelmina,✔️ [True]
8,What was the name of the German offensive in the Ardennes Forest in World War 2?,Battle of the Bulge,None,"Reasoning: Let's think step by step in order to identify the battle. We know it was a German offensive, and it took place in the...",False
9,Which battle in World War 2 marked the Japanese defeat at the hands of the United States Navy?,Battle of Midway,None,Battle of Midway,✔️ [True]


52.94

In [33]:
compiled_rag("Who was the leader of France during its occupation by Germany in World War 2?").answer

'Philippe Pétain'

In [34]:
compiled_rag("What was the name of the German defensive line in Italy during World War 2?").answer

'Gustav Line'